# Installation librairies

In [ ]:
!pip install shapely
!pip install geopy==2.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: geopy
    Found existing installation: geopy 2.4.1
    Uninstalling geopy-2.4.1:
      Successfully uninstalled geopy-2.4.1


# Import librairies

In [ ]:
import pandas as pd
import geopandas as gpd
import json
from shapely.geometry import shape, Point, Polygon, box
import folium
from folium.plugins import HeatMap
from geopy.distance import geodesic
import requests
import os
from IPython.display import display
from shapely import wkt

# DATA IMPORT:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# chemins
path = '/content/drive/MyDrive/ICU/'
path_down = '/content/drive/MyDrive/ICU/Download_data/'
path_analyse = '/content/drive/MyDrive/ICU/Analyse/'
path_carto = '/content/drive/MyDrive/ICU/Carto/'

# Importer depuis une URL API un csv vers Google drive

## Population par Quartiers

In [ ]:
# Configuration
API_URL = 'https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/recensement-population-2020-grands-quartiers-population/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B'

# Chemin complet du fichier où sauvegarder le CSV
file_path = path_down+'Densite_Pop.csv'

# Télécharger les données CSV
def download_csv(api_url, file_path):
    response = requests.get(api_url)
    response.raise_for_status()  # Assure que la requête a réussi
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f"CSV file downloaded and saved as {file_path}")

# Exécuter les fonctions
if __name__ == '__main__':
    download_csv(API_URL, file_path)

CSV file downloaded and saved as /content/drive/MyDrive/ICU/Download_data/Densite_Pop.csv


In [ ]:
# import Densite_Pop.csv
df_pop = pd.read_csv(path_down+'Densite_Pop.csv',sep=';', low_memory=False)

## Entreprise par Quartiers

In [ ]:
# Configuration
API_URL = 'https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/base-sirene-v3/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B'

# Chemin complet du fichier où sauvegarder le CSV
file_path = path_down+'Entreprise.csv'

# Télécharger les données CSV
def download_csv(api_url, file_path):
    response = requests.get(api_url)
    response.raise_for_status()  # Assure que la requête a réussi
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f"CSV file downloaded and saved as {file_path}")

# Exécuter les fonctions
if __name__ == '__main__':
    download_csv(API_URL, file_path)

CSV file downloaded and saved as /content/drive/MyDrive/ICU/Download_data/Entreprise.csv


In [ ]:
# import Entreprise.csv
df_ent = pd.read_csv(path_down+'Entreprise.csv',sep=';', low_memory=False)

# Import Dataset

In [ ]:
#import geopoint des stations
df_station = pd.read_csv(path_down+'Liste_station.csv',sep=';')
df_station = df_station[df_station['emission']!='N'][['id_numero','id_nom','geopoint']]
df_station = df_station.sort_values('id_numero').reset_index(drop=True)

# Entreprise

In [ ]:
df_ent_filter = df_ent[(df_ent["Etat administratif de l'établissement"] != "Fermé") &
               (df_ent["Date de création de l'établissement"] < "2023-12-31")]

In [ ]:
# Étape 1: Filtrer les valeurs non valides dans les coordonnées des entreprises
df_ent_filter = df_ent_filter[df_ent_filter['Géolocalisation de l\'établissement'].apply(lambda x: isinstance(x, str))]

# Convertir les coordonnées des entreprises en objets Point
df_ent_filter['geometry'] = df_ent_filter['Géolocalisation de l\'établissement'].apply(lambda x: Point(map(float, x.split(','))))

# Convertir df_filter en GeoDataFrame
gdf_filter = gpd.GeoDataFrame(df_ent_filter, geometry='geometry', crs='EPSG:4326')

# Convertir les coordonnées des stations en objets Point et les ajouter à un GeoDataFrame
df_station['geometry'] = df_station['geopoint'].apply(lambda x: Point(map(float, x.split(','))))
gdf_station = gpd.GeoDataFrame(df_station, geometry='geometry', crs='EPSG:4326')

# Transformer le CRS en un système projeté pour des calculs de distance (par exemple, EPSG:2154 pour la France)
gdf_station = gdf_station.to_crs(epsg=2154)
gdf_filter = gdf_filter.to_crs(epsg=2154)

# Étape 2: Créer des buffers de 300 mètres autour de chaque station
gdf_station['buffer'] = gdf_station['geometry'].buffer(300)

# Étape 3: Initialiser une colonne pour le nombre total d'entreprises dans le buffer
gdf_station['total_entreprise_count'] = 0

# Étape 4: Initialiser des colonnes pour chaque catégorie d'entreprise
categories = df_ent_filter[df_ent_filter["Catégorie de l'entreprise"].notna()]["Catégorie de l'entreprise"].unique()
for category in categories:
    gdf_station[f'nb_entreprises_{category}'] = 0

# Étape 5: Pour chaque station, compter le nombre d'entreprises dans le buffer et par catégorie
for idx, station in gdf_station.iterrows():
    # Sélectionner les entreprises dans le buffer
    entreprises_dans_buffer = gdf_filter[gdf_filter['geometry'].within(station['buffer'])]

    # Compter le nombre total d'entreprises dans le buffer
    gdf_station.at[idx, 'total_entreprise_count'] = entreprises_dans_buffer.shape[0]

    # Compter les entreprises par catégorie
    for category in categories:
        count = entreprises_dans_buffer[entreprises_dans_buffer['Catégorie de l\'entreprise'] == category].shape[0]
        gdf_station.at[idx, f'nb_entreprises_{category}'] = count

    # Compter le nombre d'entreprises avec une catégorie manquante (NaN)
    count_nan = entreprises_dans_buffer['Catégorie de l\'entreprise'].isna().sum()
    gdf_station.at[idx, 'nb_entreprises_NaN'] = count_nan

# Supprimer la colonne 'buffer'
gdf_station.drop(columns=['buffer'], inplace=True)

# Afficher le DataFrame final
gdf_station

# Exporter Entreprise en un fichier CSV
output_path = os.path.join(path_analyse, 'resultats_Nb_Entreprise_300m.csv')  # Spécifier le chemin de sauvegarde du fichier CSV
gdf_station.to_csv(output_path, sep=';', index=False)

print(f"DataFrame exporté en CSV à : {output_path}")

DataFrame exporté en CSV à : /content/drive/MyDrive/ICU/Analyse/resultats_Nb_Entreprise_300m.csv


# Population

In [ ]:
#Keep only column Geo Point, Geo Shape, IB_GRD_QUART and P20_POP as df_pop_filter
df_pop_filter = df_pop[['Geo Point', 'Geo Shape', 'LIB_GRD_QUART', 'P20_POP']]

In [ ]:
# Séparer la colonne 'geopoint' en latitude et longitude
df_station[['latitude', 'longitude']] = df_station['geopoint'].str.split(',', expand=True)

# Convertir les colonnes en float pour être utilisées dans les tuples
df_station['latitude'] = df_station['latitude'].astype(float)
df_station['longitude'] = df_station['longitude'].astype(float)

# Créer la liste de tuples (longitude, latitude)
stations_coords = list(zip(df_station['longitude'], df_station['latitude']))

# Convertir la colonne 'Geo Shape' de chaînes JSON en objets Python
df_pop_filter['geometry'] = df_pop_filter['Geo Shape'].apply(lambda x: shape(json.loads(x)))

# Définir le CRS initial si les données sont en EPSG:4326 (WGS84)
gdf_pop_filter = gpd.GeoDataFrame(df_pop_filter, geometry='geometry', crs="EPSG:4326")

def create_map_and_table_with_quartiers_and_population(stations_df, gdf_pop_filter):
    # Séparer la colonne 'geopoint' en latitude et longitude
    stations_df[['latitude', 'longitude']] = stations_df['geopoint'].str.split(',', expand=True)
    stations_df['latitude'] = stations_df['latitude'].astype(float)
    stations_df['longitude'] = stations_df['longitude'].astype(float)

    # Créer la liste de tuples (longitude, latitude)
    stations_coords = list(zip(stations_df['longitude'], stations_df['latitude']))

    # Convertir la colonne 'Geo Shape' de chaînes JSON en objets Python
    gdf_pop_filter.loc[:, 'geometry'] = gdf_pop_filter['Geo Shape'].apply(lambda x: shape(json.loads(x)))

    # Définir le CRS initial si les données sont en EPSG:4326 (WGS84)
    gdf_pop_filter = gpd.GeoDataFrame(gdf_pop_filter, geometry='geometry', crs="EPSG:4326")

    # Créer une carte centrée sur le premier point de la liste de stations
    m = folium.Map(location=[stations_coords[0][1], stations_coords[0][0]], zoom_start=13)

    # Liste pour stocker les résultats
    results = []

    # Boucle sur chaque station
    for index, (station_coords, station_row) in enumerate(zip(stations_coords, stations_df.itertuples(index=False))):
        # Créer une GeoSeries pour la station avec CRS EPSG:4326
        station_point = gpd.GeoSeries([Point(station_coords)], crs="EPSG:4326")

        # Convertir la station en EPSG:3857 pour calculer le buffer de 300m
        station_point_proj = station_point.to_crs(epsg=3857)

        # Créer un buffer de 300m dans le CRS projeté
        circle_300m = station_point_proj.buffer(300).iloc[0]  # Buffer autour de la station (dans le CRS projeté)

        # Convertir le cercle en EPSG:4326 pour l'affichage sur la carte
        circle_300m = gpd.GeoSeries([circle_300m], crs="EPSG:3857").to_crs(epsg=4326)

        # Initialiser la population totale à 0
        population_totale = 0

        # Liste pour les détails des quartiers intersectés
        quartiers_intersectes = []
        population_intersected = []

        # Tracer les quartiers et calculer la population dans le cercle
        for _, quartier in gdf_pop_filter.iterrows():
            quartier_shape = quartier['geometry']
            intersection = quartier_shape.intersection(circle_300m.iloc[0])  # Intersection du quartier avec le cercle

            if not intersection.is_empty:  # Si l'intersection n'est pas vide
                # Calculer la proportion de l'intersection par rapport à la surface du quartier
                proportion = intersection.area / quartier_shape.area
                # Calculer la population pour la partie du quartier à l'intérieur du cercle
                population_quartier = proportion * quartier['P20_POP']
                population_totale += population_quartier

                # Ajouter les détails du quartier intersecté
                quartiers_intersectes.append(quartier['LIB_GRD_QUART'])
                population_intersected.append(int(population_quartier))

                # Ajouter le quartier et la population à la carte
                popup_text = f"Quartier: {quartier['LIB_GRD_QUART']}<br>Population (dans cercle): {int(population_quartier)}"
                folium.GeoJson(quartier_shape, style_function=lambda x: {
                    'fillColor': 'blue', 'color': 'blue', 'weight': 2, 'fillOpacity': 0.1
                }, tooltip=popup_text).add_to(m)

        # Tracer le cercle de 300m autour de la station
        folium.GeoJson(circle_300m.iloc[0], style_function=lambda x: {
            'fillColor': 'red', 'color': 'red', 'weight': 2, 'fillOpacity': 0.3
        }).add_to(m)

        # Ajouter un marqueur pour la station
        folium.Marker(
            location=[station_coords[1], station_coords[0]],
            popup=f"Station: {station_row.id_nom}<br>Population totale dans 300m: {int(population_totale)}",
            icon=folium.Icon(color='green')
        ).add_to(m)

        # Créer une ligne pour le DataFrame
        results.append({
            'id': station_row.id_numero,
            'Nom': station_row.id_nom,
            'Population_totale_300m': int(population_totale),
            'Quartiers_intersectes': ', '.join(quartiers_intersectes),
            'Population_par_quartier': ', '.join(map(str, population_intersected))
        })

    # Créer un DataFrame à partir des résultats
    df_results = pd.DataFrame(results)

    return m, df_results

# Créer la carte et compiler les résultats dans une table
map_with_population, table_results = create_map_and_table_with_quartiers_and_population(df_station, gdf_pop_filter)

# Afficher la carte
display(map_with_population)

# Afficher le DataFrame des résultats
display(table_results)

# Exporter resultats_population_par_quartier en un fichier CSV
output_path = os.path.join(path_analyse, 'resultats_population_par_quartier.csv')  # Spécifier le chemin de sauvegarde du fichier CSV
table_results.to_csv(output_path, sep=';', index=False)

print(f"DataFrame exporté en CSV à : {output_path}")

<ipython-input-13-b39c67827711>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pop_filter['geometry'] = df_pop_filter['Geo Shape'].apply(lambda x: shape(json.loads(x)))


,id,Nom,Population_totale_300m,Quartiers_intersectes,Population_par_quartier
0,0,00-station-meteo-toulouse-valade,2115,"Capitole, Arnaud Bernard","832, 1283"
1,1,01-station-meteo-toulouse-meteopole,142,Basso-Cambo,142
2,2,02-station-meteo-toulouse-marengo,1460,Marengo-Jolimont,1460
3,3,03-station-meteo-toulouse-busca,1622,"Le Busca, Saint-Michel","1330, 292"
4,4,04-station-meteo-toulouse-ile-empalot,54,"Rangueil-C.H.U.-Facultés, Ramier, Zones d'Acti...","13, 23, 17"
...,...,...,...,...,...
59,65,65-station-meteo-toulouse-LIFE-gastou,812,"Ramier, Fer-à-Cheval","49, 763"
60,65,65-station-meteo-toulouse-LIFE-rapas,921,"Ramier, Fer-à-Cheval","38, 883"
61,66,66-station-meteo-toulouse-LIFE-est-stadium,118,Ramier,118
62,66,66-station-meteo-toulouse-LIFE-coubertin,131,"Ramier, Saint-Michel, Empalot","117, 11, 2"


DataFrame exporté en CSV à : /content/drive/MyDrive/ICU/Analyse/resultats_population_par_quartier.csv
